In [1]:
x=1

In [4]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import decoupler as dc
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import pertpy 

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
# adata = sc.read('../../../pipeline/data/multigrate_mil/hlca_healthy_ipf_embed_balanced/b2e8c62cf5_adata_both.h5ad')
# adata

In [ ]:
# hlca = sc.read('../../../../snakemake/hlca/5fold/hlca_hyperparam/data/pp/hlca_hvg_pearson.h5ad')
# adata.obsm['counts'] = hlca[adata.obs_names].X

In [ ]:
# adata.write('../../../../data_to_share/hlca_healthy_ipf_subset.h5ad')

In [ ]:
adata = sc.read('../../../../data_to_share/hlca_healthy_ipf_subset.h5ad')
adata

In [ ]:
adata.obsm['counts'].data

In [ ]:
adata.obsm['counts']

In [ ]:
import rpy2.robjects as ro

def run_de(adata):
    r=ro.r
    getwd = ro.r['getwd']
    print(getwd())
    r.source("de_full_ct.R")
    df = r.fit_model(adata)
    return df

In [ ]:
for ct in np.unique(adata.obs['transf_ann_level_3_label'].astype(str)):
    print(f'Processing {ct}...')
    adata_ct = adata[adata.obs['transf_ann_level_3_label'] == ct].copy()
    adata_ct_disease = adata_ct[adata_ct.obs['lung_condition_coarse'] == 'COPD'].copy()

    print(len(adata_ct_disease))
    
    threshold_idx = int(len(adata_ct_disease) * 0.9)
    threshold_value = sorted(adata_ct_disease.obs['cell_attn'])[threshold_idx]
    
    adata_ct_disease.obs['group'] = 'all'
    adata_ct_disease.obs.loc[adata_ct_disease.obs['cell_attn'] >= threshold_value, 'group'] = 'top'

    # adata_ct_disease.obs['sample_group'] = adata_ct_disease.obs['sample'].astype('str') + '_' + adata_ct_disease.obs['group']
    # adata_ct_disease.obs['sample_group'] = adata_ct_disease.obs['sample_group'].astype('category')

    # samples_to_exclude = []
    # for sample in np.unique(adata_ct_disease.obs[['sample_group']]):
    #     sample_len = len(adata_ct_disease[adata_ct_disease.obs['sample_group'] == sample])
    #     if sample_len < 10:
    #         samples_to_exclude.append(sample)
    
    # adata_ct_disease = adata_ct_disease[~adata_ct_disease.obs['sample_group'].isin(samples_to_exclude)].copy()

    #adata_ct_disease.X = adata_ct_disease.obsm['counts'].copy()
    adata_pb = sc.AnnData(adata_ct_disease.obsm['counts'])
    adata_pb.obs = adata_ct_disease.obs
    adata_pb.var_names = hlca.var_names

    adata_ = dc.get_pseudobulk(adata_pb, 'sample', 'group', mode='sum', min_cells=-1, min_counts=10)

    adata_r = sc.AnnData(adata_.X, obs=adata_.obs[['group', 'study', 'sample']])
    adata_r.var_names = adata_.var_names
    adata_r.obs['sample'] = adata_r.obs['sample'].str.replace('-', '_')
    adata_r.obs.index = adata_r.obs.index.str.replace('-', '_')

    adata_r = adata_r.copy()

    sc.pp.filter_genes(adata_r, min_cells=10)
    print(adata_r.X.shape)
    if ((adata_r.X.shape[0] == 0) or (adata_r.X.shape[1] == 0)):
        continue
    df = run_de(adata_r)

    df = ro.conversion.rpy2py(df)
    df.to_csv(f'{ct}.csv')